In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('weather.csv') 
data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [3]:
X = data.iloc[:,1:data.shape[1] - 1]
Y = data.iloc[:,data.shape[1] - 1]

In [4]:
def CalcEntropy(labels):
    n=len(labels)
    if n<=1:
        return 0
    counts=np.bincount(labels)
    prob=counts[np.nonzero(counts)]/n
    noc=len(prob)
    if noc<=1:
        return 0
    return -np.sum(prob*np.log(prob))/np.log(noc)

In [5]:
l=data.iloc[:,data.shape[1] - 1].map({'No':0,'Yes':1})
l

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: play, dtype: int64

In [6]:
E_S=CalcEntropy(l)

In [7]:
def GiveLabels(d,Y,string1,string2):
    labels=[]
    for i in d[string1]:
        if i==string2:
            a=pd.Series(Y[d[string1]==string2])
            labels=a.map({'No':0,'Yes':1})
    return labels

In [8]:
dict_={}

In [9]:
cl=len(X.columns)

In [10]:
for i in range(0,cl):
    dict_.update({X.columns[i]:[]})

In [11]:
for i in dict_.keys():
    for j in X[i].unique():
            dict_[i].append(CalcEntropy(GiveLabels(X,Y,i,j)))

In [12]:
dict_

{'outlook': [0.9709505944546688, 0, 0.9709505944546688],
 'temp': [1.0, 0.9182958340544894, 0.8112781244591328],
 'humidity': [0.9852281360342515, 0.5916727785823274],
 'wind': [0.8112781244591328, 1.0]}

In [13]:
def GainEach(E,d,Y,dic):
    c=X.columns
    n=len(Y)
    g=[]
    for column in c:
        col=d[column].unique()
        g1=0
        for j in range(len(col)):
            g1+=(len(GiveLabels(d,Y,column,col[j]))/n)*dic[column][j]
        g.append(E-g1)
    return g

In [14]:
gain1=GainEach(E_S,X,Y,dict_)
firstSplit=gain1.index(max(gain1))
node1=X.columns[firstSplit]
node1

'outlook'

In [15]:
da_1= data[data[node1] == data[node1].unique()[0]]
Y_1=da_1.iloc[:,data.shape[1]-1]
da_1=da_1.drop(['day','play'],axis=1)


In [16]:
dict_={}
cl=len(da_1.columns)
for i in range(0,cl):
        dict_.update({da_1.columns[i]:[]})
for i in dict_.keys():
    for j in da_1[i].unique():
            dict_[i].append(CalcEntropy(GiveLabels(da_1,Y_1,i,j))) 

In [17]:
dict_

{'outlook': [0.9709505944546688],
 'temp': [0, 1.0, 0],
 'humidity': [0, 0],
 'wind': [0.9182958340544894, 1.0]}

In [18]:
gain1=GainEach(dict_[node1][0],da_1,Y_1,dict_)
secondSplit=gain1.index(max(gain1))
node2=X.columns[secondSplit]
node2

'humidity'

In [19]:
da_1= data[data[node1] == data[node1].unique()[1]]
Y_1=da_1.iloc[:,data.shape[1]-1]
da_1=da_1.drop(['day','play'],axis=1)

In [20]:
dict_={}
cl=len(da_1.columns)
for i in range(0,cl):
        dict_.update({da_1.columns[i]:[]})
for i in dict_.keys():
    for j in da_1[i].unique():
            dict_[i].append(CalcEntropy(GiveLabels(da_1,Y_1,i,j))) 

In [21]:
dict_

{'outlook': [0], 'temp': [0, 0, 0], 'humidity': [0, 0], 'wind': [0, 0]}

In [22]:
"""
gain1=GainEach(dict_[node1][0],da_1,Y_1,dict_)
secondSplit=gain1.index(max(gain1))
node2=X.columns[secondSplit]
node2
"""

'\ngain1=GainEach(dict_[node1][0],da_1,Y_1,dict_)\nsecondSplit=gain1.index(max(gain1))\nnode2=X.columns[secondSplit]\nnode2\n'

In [23]:
da_1= data[data[node1] == data[node1].unique()[2]]
Y_1=da_1.iloc[:,data.shape[1]-1]
da_1=da_1.drop(['day','play'],axis=1)

In [24]:
dict_={}
cl=len(da_1.columns)
for i in range(0,cl):
        dict_.update({da_1.columns[i]:[]})
for i in dict_.keys():
    for j in da_1[i].unique():
            dict_[i].append(CalcEntropy(GiveLabels(da_1,Y_1,i,j))) 

In [25]:
dict_

{'outlook': [0.9709505944546688],
 'temp': [0.9182958340544894, 1.0],
 'humidity': [1.0, 0.9182958340544894],
 'wind': [0, 0]}

In [26]:
gain1=GainEach(dict_[node1][0],da_1,Y_1,dict_)
secondSplit=gain1.index(max(gain1))
node3=X.columns[secondSplit]
node3

'wind'

In [27]:
print("Thus the final decision tree will be like this..")
print("If outlook = Sunny, Humidity = High then Play tennis = No")
print("If outlook = Sunny, Humidity = Normal then Play tennis = Yes")
print("If outlook = Overcast, then Play tennis = Yes")
print("If outlook = Rain, Wind = Strong then Play tennis = No")
print("If outlook = Rain, Wind = Weak then Play tennis = Yes")

Thus the final decision tree will be like this..
If outlook = Sunny, Humidity = High then Play tennis = No
If outlook = Sunny, Humidity = Normal then Play tennis = Yes
If outlook = Overcast, then Play tennis = Yes
If outlook = Rain, Wind = Strong then Play tennis = No
If outlook = Rain, Wind = Weak then Play tennis = Yes
